## Building model for content based filtering (This is a test)

In [1]:
import pandas as pd

In [17]:
# coming to the evaluation metric I'm using RMSE

viewed = pd.read_csv('../data/posts_features.csv')
viewed

,id,category_id,title,video_link,emotions
0,11,2,Recipe for a flow state,https://video-cdn.socialverseapp.com/michael_9...,"actions, audio_elements, description, emotions..."
1,12,2,Why fit in..?,https://video-cdn.socialverseapp.com/michael_5...,"actions, audio_elements, description, emotions..."
2,13,2,Transcending Singularity,https://video-cdn.socialverseapp.com/michael_5...,"actions, audio_elements, description, emotions..."
3,14,2,Peak Performance?,https://video-cdn.socialverseapp.com/michael_4...,"actions, audio_elements, description, emotions..."
4,15,2,Our Existential Situation,https://video-cdn.socialverseapp.com/michael_b...,"actions, audio_elements, description, emotions..."
...,...,...,...,...,...
995,1332,13,"Kalsubai is the highest peak in Maharashtra, s...",https://video-cdn.socialverseapp.com/shivam_03...,"actions, audio_elements, description, emotions..."
996,1334,13,"Tap into the magic of winter. .Sometimes, all ...",https://video-cdn.socialverseapp.com/shivam_03...,"actions, audio_elements, description, emotions..."
997,1335,13,anzeige -- ️From idea to masterpiece ️with the,https://video-cdn.socialverseapp.com/shivam_03...,"actions, audio_elements, description, emotions..."
998,1336,13,"In the heart of darkness, magic is our only sh...",https://video-cdn.socialverseapp.com/shivam_03...,"actions, audio_elements, description, emotions..."


# Here user id and post id are categorical

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

# Load data
data = pd.read_csv("../data/liked_posts.csv")

In [6]:
# Feature engineering
data['liked_at'] = pd.to_datetime(data['liked_at'])
data['day_of_week'] = data['liked_at'].dt.dayofweek
data['hour_of_day'] = data['liked_at'].dt.hour
data['days_since_like'] = (data['liked_at'].max() - data['liked_at']).dt.days

In [7]:
# Encode categorical variables
le_user = LabelEncoder()
le_post = LabelEncoder()
data['user_id_encoded'] = le_user.fit_transform(data['user_id'])
data['post_id_encoded'] = le_post.fit_transform(data['post_id'])

In [8]:
# Train-test split (based on timestamp for time-aware splitting)
train = data[data['liked_at'] < "2024-01-01"]
test = data[data['liked_at'] >= "2024-01-01"]

In [9]:
# Features and target
X_train = train[['user_id_encoded', 'post_id_encoded', 'day_of_week', 'hour_of_day', 'days_since_like']]
y_train = np.ones(len(train))  # Target is 1 for liked posts
X_test = test[['user_id_encoded', 'post_id_encoded', 'day_of_week', 'hour_of_day', 'days_since_like']]
y_test = np.ones(len(test))

In [10]:
# Train a model
model = GradientBoostingRegressor()
model.fit(X_train, y_train)

GradientBoostingRegressor()

In [11]:
# Predict and evaluate
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse}")

RMSE: 0.0


In [12]:
# Recommend top-N posts for each user in the test set
test['predicted_score'] = y_pred
top_n_recommendations = test.sort_values(by='predicted_score', ascending=False).groupby('user_id').head(5)
print(top_n_recommendations)

       id  post_id  user_id            liked_at  day_of_week  hour_of_day  \
120   212      631      102 2024-01-01 02:30:53            0            2   
711  1242     1066      672 2024-06-27 17:08:18            3           17   
700  1231     1053      669 2024-06-26 18:57:31            2           18   
701  1232     1065      680 2024-06-27 07:24:45            3            7   
702  1233     1066      680 2024-06-27 07:24:57            3            7   
..    ...      ...      ...                 ...          ...          ...   
400   655     1001      195 2024-04-23 17:20:27            1           17   
401   656     1002      195 2024-04-23 17:20:29            1           17   
402   657     1003      195 2024-04-23 17:20:30            1           17   
403   658     1004      195 2024-04-23 17:20:32            1           17   
999  1628     1110      920 2024-08-10 18:43:07            5           18   

     days_since_like  user_id_encoded  post_id_encoded  predicted_score  
1

C:\Users\vishw\AppData\Local\Temp\ipykernel_33600\2815537752.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['predicted_score'] = y_pred
